In [ ]:
'''
TODO:
+ implement path class
+ root node class
+ remove spaghetti
'''

In [3]:
from PDBReader import atom_positions_from_pdb
from test import *
from octree import *

In [4]:
test_classification(1)
test_node_counting()

test passed
test passed


In [5]:
atom_positions = atom_positions_from_pdb("2erk.pdb")

In [7]:
tree = Octree(atom_positions, 5).tree